In [1]:
from nltk.corpus import stopwords

In [3]:
sw = stopwords.words("english")
print "There are", len(sw), "stopwords"

There are 153 stopwords


In [4]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
stemmer.stem("responsiveness")

u'respons'

In [5]:
print stemmer.stem("responsivity")
print stemmer.stem("unresponsive")

respons
unrespons


In [6]:
import sys
sys.path.append('../tools/')

In [9]:
run ../tools/parse_out_email_text.py



Hi Everyone  If you can read this message youre properly using parseOutText  Please proceed to the next part of the project



In [11]:
# Deploying Stemming
from nltk.stem.snowball import SnowballStemmer
import string

def parseOutText(f, stemming = False):
    """ given an opened email file f, parse out all text below the
        metadata block at the top
        (in Part 2, you will also add stemming capabilities)
        and return a string that contains all the words
        in the email (space-separated) 
        
        example use case:
        f = open("email_file_name.txt", "r")
        text = parseOutText(f)
        
        """


    f.seek(0)  ### go back to beginning of file (annoying)
    all_text = f.read()

    ### split off metadata
    content = all_text.split("X-FileName:")
    words = ""
    if len(content) > 1:
        ### remove punctuation
        text_string = content[1].translate(string.maketrans("", ""), string.punctuation)

        if stemming:
            ### split the text string into individual words, stem each word,
            ### and append the stemmed word to words (make sure there's a single
            ### space between each stemmed word)
            text_split = text_string.split()
            stemmer = SnowballStemmer("english")
            for word in text_split:
                words += stemmer.stem(word) + ' '
        else:
            ### project part 2: comment out the line below
            words = text_string

    return words

In [12]:
ff = open("test_email.txt", "r")
text = parseOutText(ff, stemming=True)

In [13]:
print text

hi everyon if you can read this messag your proper use parseouttext pleas proceed to the next part of the project 


## Clean Away "Signature Words"
Parsing emails from Chris and Sara, getting stemmed text strings and then removing signature words

In [ ]:
import os
import pickle
import re

In [33]:
"""
    Starter code to process the emails from Sara and Chris to extract
    the features and get the documents ready for classification.

    The list of all the emails from Sara are in the from_sara list
    likewise for emails from Chris (from_chris)

    The actual documents are in the Enron email dataset, which
    you downloaded/unpacked in Part 0 of the first mini-project. If you have
    not obtained the Enron email corpus, run startup.py in the tools folder.

    The data is stored in lists and packed away in pickle files at the end.
"""

def process_emails(fullSet=False):

    with open("from_sara.txt", "r") as from_sara, open("from_chris.txt", "r") as from_chris:
        from_data = []
        word_data = []

        ### temp_counter is a way to speed up the development--there are
        ### thousands of emails from Sara and Chris, so running over all of them
        ### can take a long time
        ### temp_counter helps you only look at the first 200 emails in the list so you
        ### can iterate your modifications quicker
        temp_counter = 0

        name_codes = {"sara":0,"chris":1}

        for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
            for path in from_person:
                ### only look at first 200 emails when developing
                ### once everything is working, remove this line to run over full dataset
                temp_counter += 1
                if fullSet or temp_counter < 200:
                    #Adding extra level because maildir is not in my project folder
                    path = os.path.join('../..', path[:-1])
                    print path
                    with open(path, "r") as email:
                        email_text = parseOutText(email, stemming=True)

                        for replace_word in ["sara", "shackleton", "chris", "germani"]:
                            email_text = email_text.replace(replace_word,'')
                        ### use str.replace() to remove any instances of the words
                        ### ["sara", "shackleton", "chris", "germani"]

                        ### append the text to word_data
                        word_data.append(email_text)

                        ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
                        from_data.append(name_codes[name])


    print "emails processed"
    return word_data, from_data

pickle.dump( word_data, open("your_word_data.pkl", "w") )
pickle.dump( from_data, open("your_email_authors.pkl", "w") )

In [32]:
print(word_data[152])

tjonesnsf stephani and sam need nymex calendar 


## TfIdf It
Removing English stopwords from emails and then transforming into a tf-idf matrix

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
word_data, from_data = processEmails(fullset=True)

In [30]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit_transform(word_data)

<17578x38757 sparse matrix of type '<type 'numpy.float64'>'
	with 1078821 stored elements in Compressed Sparse Row format>

In [31]:
print "The number of unique words in the emails is", len(tfidf.get_feature_names())

38757

## Accessing TfIdf Features
Getting a particular word from the list of unique words

In [37]:
word_list = tfidf.get_feature_names()
print word_list[34597]

stephaniethank
